In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
print(spacex_df.head())
spacex_df1=pd.DataFrame(spacex_df)
spacex_df['Launch Site'].unique()

In [ ]:


# Read the airline data into pandas dataframe

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                  dcc.Dropdown(id='site-dropdown',
                options=[
                    {'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value':'KSC LC-39A' },
                    {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40' },
                ],
                value='ALL',
                placeholder="place holder here",
                searchable=True
                ),
                                
                                
html.Br(),

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
# Function decorator to specify function input and output

    #else:

html.Div(dcc.Graph(id='success-pie-chart')),
html.Br(),

html.P("Payload range (Kg):"),
# TASK 3: Add a slider to select payload range
html.Div(dcc.RangeSlider(0,10000,1000, value=[min_payload, max_payload],id='payload-slider')),

# TASK 4: Add a scatter chart to show the correlation between payload and launch success
html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    
    if entered_site == 'ALL':
        filtered_df = spacex_df.groupby(['Launch Site'])['class'].sum().to_frame()
        filtered_df= filtered_df.reset_index()
        #print(filtered_df)
        fig = px.pie(filtered_df, values='class',names='Launch Site',title='Pie Chart')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]["class"].value_counts().to_frame()
        filtered_df["status"]=["Failed","Success"]
        fig = px.pie(filtered_df, values='class', 
        names='status', 
        title='Pie Chart')
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')
             )

def get_scatter_chart(entered_site1,payload_range):    
    if entered_site1 == 'ALL':
        filtered_df1 = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_range[0])
                               & (spacex_df['Payload Mass (kg)']<=payload_range[1])
                                ]
        print(filtered_df1)
        fig1 = px.scatter(filtered_df1,x="Payload Mass (kg)",y="class", color='Booster Version Category')
    else:
        filtered_df1 = spacex_df[ (spacex_df['Launch Site']==entered_site1)
                               & (spacex_df['Payload Mass (kg)']>=payload_range[0])
                               & (spacex_df['Payload Mass (kg)']<=payload_range[1])
                                ]
        print(filtered_df1)
        fig1 = px.scatter(filtered_df1,x="Payload Mass (kg)",y="class", color='Booster Version Category')
    
    return fig1

# Run the app
if __name__ == '__main__':
    app.run_server()